In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test_path = '/kaggle/input/100-bird-species/test'
train_path = '/kaggle/input/100-bird-species/train'
os.listdir(test_path)
os.listdir(train_path)

In [ ]:
len(os.listdir(train_path))

In [ ]:
# Count images for each species
def cntSamples(directory):
    specs = []
    for root, dirs, files in os.walk(directory, topdown=True):
        dirs.sort()
        for name in dirs:
            if name not in specs:
                specs.append(name)

    # file counts for each species 
    nums = []
    for b in specs:
        path = os.path.join(directory,b)
        num_files = len(os.listdir(path))
        nums.append(num_files)
 
    # Create Dictionary
    adict = {specs[i]:nums[i] for i in range(len(specs))}
    return adict

In [ ]:
testDict =  cntSamples(test_path)
trainDict = cntSamples(train_path)

In [ ]:
from IPython.core.display import display, HTML
train_tbl = pd.DataFrame.from_dict(trainDict, 
                orient='index', dtype=None, columns=['Images'])

num_classes = len(trainDict)
label_index = list(range(num_classes))
print(num_classes)
train_tbl.insert(0,'Label Index',label_index, True)

display(HTML(train_tbl.to_html()))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_gen = ImageDataGenerator(rotation_range=20, # rotate the image 20 degrees
                               width_shift_range=0.10, # Shift the pic width by a max of 5%
                               height_shift_range=0.10, # Shift the pic height by a max of 5%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.1, # Shear means cutting away part of the image (max 10%)
                               zoom_range=0.1, # Zoom in by 10% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [ ]:
image_gen.flow_from_directory(train_path)
image_gen.flow_from_directory(test_path)


Building model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
image_shape = (224, 224, 3)
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))
model.add(Dense(200))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=4)

batch_size = 30
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                              class_mode='categorical')
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='categorical',shuffle=False)

#train_image_gen.class_indices

In [ ]:
train_image_gen.class_indices

In [ ]:
test_image_gen.class_indices

In [ ]:
import warnings
warnings.filterwarnings('ignore')
results = model.fit_generator(train_image_gen,epochs=100,
                              validation_data=test_image_gen,
                             callbacks=[early_stop])

In [ ]:
import pandas as pd
model.save('100_bird_species.h5')
losses = pd.DataFrame(model.history.history)
losses[['loss','val_loss']].plot()


In [ ]:
losses[['accuracy','val_accuracy']].plot()

In [ ]:
model.evaluate_generator(test_image_gen)
from tensorflow.keras.preprocessing import image
pred_probabilities = model.predict_generator(test_image_gen)
pred_probabilities

In [ ]:
predictions = pred_probabilities > 0.5
predictions

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(test_image_gen.classes,predictions))

In [ ]:
confusion_matrix(test_image_gen.classes,predictions)
#os.listdir(test_path+'\\def_front')[0]
eastern_rosella_image_cell = test_path+'\\EASTERN ROSELLA\1.jpg
casting_image_cell = test_path+'\YELLOW HEADED BLACKBIRD\1.jpgjpeg'

In [ ]:
eastern_rosella_image_test = image.load_img(eastern_rosella_image_cell,target_size=image_shape)
casting_image_cell_test = image.load_img(casting_image_cell,target_size=image_shape)
eastern_rosella_image_test = image.img_to_array(eastern_rosella_image_test)
casting_image_cell_test = image.img_to_array(casting_image_cell_test)

eastern_rosella_image_test = np.expand_dims(eastern_rosella_image_test, axis=0)
casting_image_cell_test = np.expand_dims(casting_image_cell_test, axis=0)
casting_image_cell_test.shape

In [ ]:
model.predict(eastern_rosella_image_test)


In [ ]:
model.predict(casting_image_cell_test)